In [1]:
import pandas  as pd 
from itertools import product

Estado actual: 
* Pandas es muy lento para lo que quiero hacer
* Trabajo actual: separar los desempates en: Primero wins, despues rounds, despues mapas (probablemente no se implemente)
* Potenciales soluciones: Cambiar la manera de armar la tabla a Numpy (deberia ser lo mas rapido por mucho) Ventajas: velocidad y puedo evaluar por filas los resultados 

-------------------------------------------------------------------------

Arranco casi de 0, problemas con la unicacion de memoria de los objetos y se armo un re quilombo. 
Intento 2 diseño enfocado tabla final que tengo y estructurar desde ese punto hacia atras. 



In [2]:
#Esto es raro, siento que estoy dando vueltas en circulo con este diccionario creando los objetos y volviendo a usarlos 
#Lo termino usando con un zip, lo que puede ser peligroso con el orden y las keys
teamNameScores = {
                "name": ["Kru","Lev","100T","mibr","NRG","EG","FUR","LOUD", "2G","GAMERS2","C9","SEN"],
                "wins": [6,9,5,1,4,4,3,4,7,00,6,6],
                "loss":[4,1,5,9,6,6,7,6,3,00,4,4],
                "matchWins" : [15,19,12,3,10,10,8,11,15,00,13,13],
                "matchLoss" : [10,4,11,18,13,15,15,14,8,00,13,8],
                "roundWin":[287,296,255,168,238,241,195,249,249,00,270,249],
                "roundLoss":[247,204,232,257,237,281,270,275,214,00,276,214]
                
                  }

roundMatches = {
                "week11" : [["Kru","Lev"], ["100T","mibr"],["NRG","EG"], ["FUR","LOUD"],["2G","GAMERS2"],["C9","SEN"]],
                "week12" : [["Kru","100T"], ["Lev","mibr"],["NRG","FUR"], ["EG","LOUD"],["C9","GAMERS2"],["2G","SEN"]]
                }

In [3]:

hierarchy_dict = {
    team: {
        "wins": teamNameScores["wins"][i],
        "loss": teamNameScores["loss"][i],
        "matchWins": teamNameScores["matchWins"][i],
        "matchLoss": teamNameScores["matchLoss"][i],
        "roundWin": teamNameScores["roundWin"][i],
        "roundLoss": teamNameScores["roundLoss"][i],
    }
    for i, team in enumerate(teamNameScores["name"])
}

In [4]:
def run_round(matches, teams, previous_results=None):
    if previous_results is None:
        previous_results = [
            {   
                "UP": 1,
                "Team": team,
                "wins": data["wins"],
                "matchWins": data["matchWins"],
                "matchLoss": data["matchLoss"],
            }
            for team, data in teams.items()
        ]

    prev_df = pd.DataFrame(previous_results)
    grouped_results = prev_df.groupby("UP")

    match_outcomes = list(product([0, 1, 2, 3], repeat=len(matches)))
    new_results = []

    result_mapping = {
        0: (0, 2),  # pierde 0-2
        1: (1, 2),  # pierde 1-2
        2: (2, 1),  # gana 2-1
        3: (2, 0),  # gana 2-0
    }

    for up, group in grouped_results:

        simul_teams = {
            team_data["Team"]: {
                "wins": team_data["wins"],
                "matchWins": team_data["matchWins"],
                "matchLoss": team_data["matchLoss"],
            }
            for _, team_data in group.iterrows()
        }

        for combination_index, combination in enumerate(match_outcomes, start=1):

            temp_teams = {team: data.copy() for team, data in simul_teams.items()}

            for match_index, result in enumerate(combination):
                team1, team2 = matches[match_index]
                match_wins, match_loss = result_mapping[result]

                if result in [2, 3]:
                    temp_teams[team1]["wins"] += 1
                    temp_teams[team1]["matchWins"] += match_wins
                    temp_teams[team1]["matchLoss"] += match_loss
                    temp_teams[team2]["matchWins"] += match_loss
                    temp_teams[team2]["matchLoss"] += match_wins
                else:
                    temp_teams[team2]["wins"] += 1
                    temp_teams[team2]["matchWins"] += match_wins
                    temp_teams[team2]["matchLoss"] += match_loss
                    temp_teams[team1]["matchWins"] += match_loss
                    temp_teams[team1]["matchLoss"] += match_wins


            new_up = f"{up}-{combination_index}"

            for team, data in temp_teams.items():
                new_results.append(
                    {
                        "UP": new_up,
                        "Team": team,
                        "wins": data["wins"],
                        "matchWins": data["matchWins"],
                        "matchLoss": data["matchLoss"],
                    }
                )

    return new_results


In [5]:
teams = hierarchy_dict
matches = (("Kru","Lev"), ("100T","mibr"),("NRG","EG"), ("FUR","LOUD"),("2G","GAMERS2"),("C9","SEN"))
matches2 = [["Kru","100T"], ["Lev","mibr"],["NRG","FUR"], ["EG","LOUD"],["C9","GAMERS2"],["2G","SEN"]]


In [6]:
match1_results = run_round(matches, teams)

In [8]:
len(match1_results)/12

4096.0

In [47]:
round1_df = pd.DataFrame(match1_results)

In [9]:
match2_results = run_round(matches, teams, match1_results)

In [11]:
len(match2_results)/12

4096.0